#3

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%reset-f
import numpy as np
import pandas as pd
import random

In [ ]:
#randomly shuffling the data
def rand(data):
  np.random.shuffle(data)
  return data

#Running all functions invloved in preprocessing of the dataset
def preprocess(data):
  data=df.values
  data=rand(data)
  data=data.T
  X_tr,X_val,X_te,Y_tr,Y_val,Y_te=data_split(data,0.7,0.1)
  X_tr,X_val,X_te=norm(X_tr,X_val,X_te)
  Y_tr=hot_encoding(Y_tr,3)
  Y_val=hot_encoding(Y_val,3)
  Y_te=hot_encoding(Y_te,3)
  return X_tr,X_val,X_te,Y_tr,Y_val,Y_te

#splitting dataset into training, validation and testing
def data_split(data,train_ratio,val_ratio):
  X=data[:data.shape[0]-1,:]
  Y=data[data.shape[0]-1:,:]
  X_tr=X[:,:int(train_ratio*X.shape[1])]
  Y_tr=Y[:,:int(train_ratio*Y.shape[1])]
  X_val=X[:,int(train_ratio*X.shape[1]):int((train_ratio+val_ratio)*X.shape[1])]
  Y_val=Y[:,int(train_ratio*Y.shape[1]):int((train_ratio+val_ratio)*Y.shape[1])]
  X_te=X[:,int((train_ratio+val_ratio)*X.shape[1]):]
  Y_te=Y[:,int((train_ratio+val_ratio)*Y.shape[1]):]
  return X_tr,X_val,X_te,Y_tr,Y_val,Y_te

#hot-encoding the class labels
def hot_encoding(Y,n_Y):
  Y_new=np.zeros((n_Y,Y.shape[1]))
  for i in range(Y.shape[1]):
    Y_new[int(Y[0,i])-1][i]=1
  return Y_new

#normalising the data
def norm(X_tr,X_val,X_te):
  mean=X_tr.mean(axis=1)
  std=X_tr.std(axis=1)
  for i in range(X_tr.shape[0]):
    X_tr[i:i+1,:]=(X_tr[i:i+1,:]-mean[i])/std[i]
    X_val[i:i+1,:]=(X_val[i:i+1,:]-mean[i])/std[i]
    X_te[i:i+1,:]=(X_te[i:i+1,:]-mean[i])/std[i]
  return X_tr,X_val,X_te

def sigmoid(Z):
  A=1/(1+np.exp(-Z))
  return A

def tan_hyperbolic(Z):
  A=(np.exp(Z)-np.exp(-Z))/(np.exp(Z)+np.exp(-Z))
  return A

def relu(Z):
  return(np.maximum(0,Z))

#initializing all the weights and biases
def init_parameters(n_X,n_h1,n_h2,n_Y):
  W1=np.random.randn(n_h1,n_X)
  b1=np.zeros((n_h1,1))
  W2=np.random.randn(n_h2,n_h1)
  b2=np.zeros((n_h2,1))
  W3=np.random.randn(n_Y,n_h2)
  b3=np.zeros((n_Y,1))
  parameters={"W1":W1,
              "b1":b1,
              "W2":W2,
              "b2":b2,
              "W3":W3,
              "b3":b3}
  return parameters

#calculating forward propagation for a single iteration and returning the value of the neurons in the hidden and output layer
def forward_propagation(X,parameters,activation):
  Z1=np.dot(parameters["W1"],X)+parameters["b1"]
  if activation=="sigmoid":
    A1=sigmoid(Z1)
  elif activation=="tan_hyperbolic":
    A1=tan_hyperbolic(Z1)
  elif activation=="relu":
    A1=relu(Z1)
  Z2=np.dot(parameters["W2"],A1)+parameters["b2"]
  if activation=="sigmoid":
    A2=sigmoid(Z2)
  elif activation=="tan_hyperbolic":
    A2=tan_hyperbolic(Z2)
  elif activation=="relu":
    A2=relu(Z2)
  Z3=np.dot(parameters["W3"],A2)+parameters["b3"]
  Y_hat=sigmoid(Z3)
  nodes={"Z1":Z1,
         "A1":A1,
         "Z2":Z2,
         "A2":A2,
         "Z3":Z3,
         "Y_hat":Y_hat}
  return nodes

#computing crossentropy cost
def Cost(Y_hat,Y):
  epsilon=1e-10
  m=Y.shape[1]
  cost=(-1/m)*np.sum(Y*np.log(Y_hat+epsilon)+(1-Y)*np.log(1-Y_hat+epsilon)) 
  return cost

In [ ]:
#calculating backward propagation for a single iteration and returning the value of the gradient of each neuron
def back_propagation(X,Y,parameters,nodes,activation,lam):
  m=X.shape[1]
  dZ3=nodes["Y_hat"]-Y
  dW3=(1/m)*np.dot(dZ3,nodes["A2"].T)+(lam/m)*parameters["W3"]
  db3=(1/m)*np.sum(dZ3,axis=1,keepdims=True)
  dA2=np.dot(parameters["W3"].T,dZ3)
  if activation=="sigmoid":
    dZ2=dA2*nodes["A2"]*(1-nodes["A2"])
  elif activation=="tan_hyperbolic":
    dZ2=dA2*(1-nodes["A2"]**2)
  elif activation=="relu":
    dZ2=dA2*np.heaviside(nodes["Z2"],0)
  dW2=(1/m)*np.dot(dZ2,nodes["A1"].T)+(lam/m)*parameters["W2"]
  db2=(1/m)*np.sum(dZ2,axis=1,keepdims=True)
  dA1=np.dot(parameters["W2"].T,dZ2)
  if activation=="sigmoid":
    dZ1=dA1*nodes["A1"]*(1-nodes["A1"])
  elif activation=="tan_hyperbolic":
    dZ1=dA1*(1-nodes["A1"]**2)
  elif activation=="relu":
    dZ1=dA1*np.heaviside(nodes["Z1"],0)
  dW1=(1/m)*np.dot(dZ1,X.T)+(lam/m)*parameters["W1"]
  db1=(1/m)*np.sum(dZ1,axis=1,keepdims=True)
  grads={"dW1":dW1,
         "db1":db1,
         "dW2":dW2,
         "db2":db2,
         "dW3":dW3,
         "db3":db3}
  return grads

#updating weights and biases 
def gradient_descent(parameters,grads,learning_rate):
  parameters["W1"]=parameters["W1"]-learning_rate*grads["dW1"]
  parameters["b1"]=parameters["b1"]-learning_rate*grads["db1"]
  parameters["W2"]=parameters["W2"]-learning_rate*grads["dW2"]
  parameters["b2"]=parameters["b2"]-learning_rate*grads["db2"]
  parameters["W3"]=parameters["W3"]-learning_rate*grads["dW3"]
  parameters["b3"]=parameters["b3"]-learning_rate*grads["db3"]
  return parameters

def accuracy(Y_hat,Y):
  a=np.abs(Y_hat-Y)
  num_mistakes=(np.sum(a))/2
  acc=100*(Y.shape[1]-num_mistakes)/Y.shape[1]
  return acc

def return_max(Y_hat):
  for i in range(Y_hat.shape[1]):
    if Y_hat[0][i]>Y_hat[1][i] and Y_hat[0][i]>Y_hat[2][i]:
      Y_hat[0][i]=1
      Y_hat[1][i]=0
      Y_hat[2][i]=0
    elif Y_hat[1][i]>Y_hat[0][i] and Y_hat[1][i]>Y_hat[2][i]:
      Y_hat[0][i]=0
      Y_hat[1][i]=1
      Y_hat[2][i]=0
    else:
      Y_hat[0][i]=0
      Y_hat[1][i]=0
      Y_hat[2][i]=1
  return Y_hat

def model_train(X,Y,n_h1,n_h2,activation,lam):
  parameters=init_parameters(X.shape[0],n_h1,n_h2,Y.shape[0])
  for i in range(1000):
    nodes=forward_propagation(X,parameters,activation)
    grads=back_propagation(X,Y,parameters,nodes,activation,lam)
    parameters=gradient_descent(parameters,grads,0.1)
    if i%50==0:
      Y_hat=return_max(nodes["Y_hat"])
      if accuracy(Y_hat,Y)==100:
        return parameters,nodes
  Y_hat=return_max(nodes["Y_hat"])
  return parameters,nodes

In [ ]:
def model_predict(X,Y,parameters,activation):
  nodes=forward_propagation(X,parameters,activation)
  Y_hat=nodes["Y_hat"]
  Y_hat=return_max(Y_hat) 
  return accuracy(Y_hat,Y)

In [ ]:
df=pd.read_excel("/content/drive/MyDrive/ML/data5.xlsx")

data=df.values
for h in range(3,6):
  sum_t=0
  sum_r=0
  sum_s=0
  tsum_t=0
  tsum_r=0
  tsum_s=0
  for n in range(5,10):
    np.random.seed(n)
    X_tr,X_val,X_te,Y_tr,Y_val,Y_te=preprocess(data)
    parameters_t,nodes_t=model_train(X_tr,Y_tr,5,h,"tan_hyperbolic",0)
    parameters_r,nodes_r=model_train(X_tr,Y_tr,5,h,"relu",0)
    parameters_s,nodes_s=model_train(X_tr,Y_tr,5,h,"sigmoid",0)
    tacc_t=model_predict(X_te,Y_te,parameters_t,"tan_hyperbolic")
    tacc_r=model_predict(X_te,Y_te,parameters_r,"relu")
    tacc_s=model_predict(X_te,Y_te,parameters_s,"sigmoid")
    acc_t=model_predict(X_val,Y_val,parameters_t,"tan_hyperbolic")
    acc_r=model_predict(X_val,Y_val,parameters_r,"relu")
    acc_s=model_predict(X_val,Y_val,parameters_s,"sigmoid")
    sum_t=sum_t+acc_t
    sum_r=sum_r+acc_r
    sum_s=sum_s+acc_s
    tsum_t=tsum_t+tacc_t
    tsum_r=tsum_r+tacc_r
    tsum_s=tsum_s+tacc_s
  
  print("overall validation accuracy(tan_hyperbolic) of",h ,":",sum_t/5,"%")
  print("overall validation accuracy(relu) of",h ,":", sum_r/5,"%")
  print("overall validation accuracy of(sigmoid)",h ,":", sum_s/5,"%")
  print("overall testing accuracy(tan_hyperbolic) of",h ,":",tsum_t/5,"%")
  print("overall testing accuracy(relu) of",h ,":", tsum_r/5,"%")
  print("overall testing accuracy of(sigmoid)",h ,":", tsum_s/5,"%")

overall validation accuracy(tan_hyperbolic) of 3 : 95.23809523809524 %
overall validation accuracy(relu) of 3 : 84.76190476190476 %
overall validation accuracy of(sigmoid) 3 : 87.61904761904762 %
overall testing accuracy(tan_hyperbolic) of 3 : 91.42857142857143 %
overall testing accuracy(relu) of 3 : 84.28571428571429 %
overall testing accuracy of(sigmoid) 3 : 89.52380952380952 %
overall validation accuracy(tan_hyperbolic) of 4 : 95.23809523809524 %
overall validation accuracy(relu) of 4 : 91.42857142857143 %
overall validation accuracy of(sigmoid) 4 : 86.66666666666666 %
overall testing accuracy(tan_hyperbolic) of 4 : 90.47619047619048 %
overall testing accuracy(relu) of 4 : 90.47619047619047 %
overall testing accuracy of(sigmoid) 4 : 88.0952380952381 %
overall validation accuracy(tan_hyperbolic) of 5 : 92.38095238095238 %
overall validation accuracy(relu) of 5 : 91.42857142857143 %
overall validation accuracy of(sigmoid) 5 : 90.47619047619048 %
overall testing accuracy(tan_hyperbolic

In [ ]:
sum_val=0
sum_te=0
for n in range(5):
  np.random.seed(n)
  X_tr,X_val,X_te,Y_tr,Y_val,Y_te=preprocess(data)
  parameters_r,nodes_r=model_train(X_tr,Y_tr,5,4,"relu",0) 
  acc_val=model_predict(X_val,Y_val,parameters_r,"relu")
  acc_te=model_predict(X_te,Y_te,parameters_r,"relu")
  sum_val=sum_val+acc_val
  sum_te=sum_te+acc_te
print("overall validation accuracy of:",sum_val/5,"%")  
print("overall testing accuracy of:",sum_te/5,"%")

overall validation accuracy of: 94.28571428571429 %
overall testing accuracy of: 90.95238095238096 %
